# Llama Stack Quick Start — MCP Option (Optional)

This notebook contains **Option B: MCP tool** only. Use it when the Llama Stack MCP adapter is ready. The main quickstart uses client-side tools only.

**Prerequisites:** Same as the main quickstart (Section 1–2: install deps, import libs, define `get_weather` is not needed here). Run the **MCP server** below, then **connect and create the agent** with MCP tools. MCP tools are **invoked by the Llama Stack Server (llama-server)**; the MCP server URL must be reachable from where the server runs.

## Option B: MCP tool

Run an MCP server that exposes a weather query tool (same capability as the client-side `get_weather`, via MCP). This example uses **Streamable HTTP** (single `/mcp` endpoint; SSE is deprecated). The server is registered with Llama Stack in the next section. *Requires the Llama Stack Server to have `tool_runtime` with the `model-context-protocol` provider.*

In [ ]:
# Start the MCP server in a separate process
import os
from multiprocessing import Process

def _run_mcp_weather_server():
    import logging
    logging.basicConfig(level=logging.DEBUG, format='%(name)s %(levelname)s: %(message)s')
    logging.getLogger("mcp").setLevel(logging.DEBUG)
    from urllib.parse import quote
    import requests
    from mcp.server.fastmcp import FastMCP
    mcp = FastMCP("demo-weather", host="0.0.0.0", port=8002)
    @mcp.tool()
    def get_weather_mcp(city: str) -> str:
        """Get current weather information for a specified city.

        Uses the wttr.in free weather API to fetch weather data.

        :param city: City name, e.g., Beijing, Shanghai, Paris
        :returns: Dictionary containing weather information including city, temperature and humidity
        """
        try:
            encoded_city = quote(city)
            url = f"https://wttr.in/{encoded_city}?format=j1"
            r = requests.get(url, timeout=10)
            r.raise_for_status()
            data = r.json()
            cur = data["current_condition"][0]
            return f"City: {city}, Temperature: {cur['temp_C']}°C, Humidity: {cur['humidity']}%"
        except Exception as e:
            return f"Error: {e}"
    # streamable-http: single endpoint; use transport="sse" and /sse if server only supports legacy SSE
    mcp.run(transport="streamable-http")

mcp_process = Process(target=_run_mcp_weather_server, daemon=True)
mcp_process.start()
import socket
# Prefer env so Llama Stack Server can reach this URL
MCP_SERVER_URL = os.getenv("MCP_SERVER_URL")
if not MCP_SERVER_URL:
    _host = socket.gethostbyname(socket.gethostname())
    if _host.startswith("127."):
        _host = os.getenv("MCP_SERVER_HOST", "127.0.0.1")
    MCP_SERVER_URL = f"http://{_host}:8002/mcp"
os.environ["MCP_SERVER_URL"] = MCP_SERVER_URL
print(f"✓ MCP server running at {MCP_SERVER_URL} (Streamable HTTP, tool: get_weather_mcp, bind 0.0.0.0:8002)")

## Connect to Server and Create Agent (MCP tools)

Register the MCP tool group and create an agent that uses it.

In [ ]:
from llama_stack_client import LlamaStackClient, Agent
from llama_stack_client.lib.agents.event_logger import AgentEventLogger

base_url = os.getenv('LLAMA_STACK_URL', 'http://localhost:8321')
client = LlamaStackClient(base_url=base_url)

models = client.models.list()
llm_model = next(
    (m for m in models
        if m.custom_metadata and m.custom_metadata.get('model_type') == 'llm'),
    None
)
if not llm_model:
    raise Exception('No LLM model found')
model_id = llm_model.id

MCP_TOOLGROUP_ID = "mcp::demo-weather"
mcp_server_url = os.getenv("MCP_SERVER_URL", "http://127.0.0.1:8002/mcp")
client.toolgroups.register(
    toolgroup_id=MCP_TOOLGROUP_ID,
    provider_id="model-context-protocol",
    mcp_endpoint={"uri": mcp_server_url},
)
agent_tools = [{"type": "mcp", "server_label": MCP_TOOLGROUP_ID, "server_url": mcp_server_url}]

agent = Agent(
    client,
    model=model_id,
    instructions='You are a helpful weather assistant. When users ask about weather, use the weather tool to query and answer.',
    tools=agent_tools,
)
print('Agent created with MCP weather tool')

### Troubleshooting (MCP / 400 error)

If you see **400 - messages[3]: invalid type: sequence, expected a string**: the inference backend often expects message `content` to be a string, but the server may send tool-turn content as an array. This is a message-format compatibility issue between the server and the backend, **not caused by SSE/Streamable HTTP**. You can:
- Use the main quickstart with **client-side tool** (Option A) instead, or
- Use **stdio** for MCP (configure the server's `tool_runtime` with `command`/`args` so the server spawns the MCP process; no HTTP URL needed), or
- Check your Llama Stack Server and inference backend docs for tool message format compatibility.

### Stop the MCP server

In [ ]:
if 'mcp_process' in globals() and mcp_process.is_alive():
    mcp_process.terminate()
    mcp_process.join(timeout=2)
    print("✓ MCP server process stopped.")
else:
    print("MCP server process is not running or has already stopped.")